# PyTorch Training and using checkpointing on SageMaker Managed Spot Training
The example here is almost the same as [PyTorch Cifar10 local training](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/pytorch_cnn_cifar10/pytorch_local_mode_cifar10.ipynb).

This notebook tackles the exact same problem with the same solution, but it has been modified to be able to run using SageMaker Managed Spot infrastructure. SageMaker Managed Spot uses [EC2 Spot Instances](https://aws.amazon.com/ec2/spot/) to run Training at a lower cost.

Please read the original notebook and try it out to gain an understanding of the ML use-case and how it is being solved. We will not delve into that here in this notebook.

In [1]:


# base_job_name = 'Select-Turn/PPTOD-base-Random/mwz20'
# base_job_name = 'Select-Turn/PPTOD-base-MaxEntropy/mwz20'
base_job_name = 'Select-Turn/PPTOD-base-LeastConfidence/mwz20'

# base_job_name = 'Train/PPTOD-base-LastTurn/mwz20'
# base_job_name = 'Train/PPTOD-base-Random/mwz20'

In [2]:
import sagemaker
import uuid
import random

sagemaker_session = sagemaker.Session()
print('SageMaker version: ' + sagemaker.__version__)

# bucket = sagemaker_session.default_bucket()
bucket = 'prod-tpgt-knowledge-lake-sandpit-v1'
prefix = 'zihan-research/dialogue-state-tracking/PPTOD-AL'

role = sagemaker.get_execution_role()
# checkpoint_suffix = str(uuid.uuid4())[:8]
checkpoint_s3_path = f's3://{bucket}/{prefix}/{base_job_name}/checkpoints'
# checkpoint_s3_path = 's3://{}/checkpoint-{}'.format(bucket, checkpoint_suffix)

print('Checkpointing Path: {}'.format(checkpoint_s3_path))

SageMaker version: 2.86.2
Checkpointing Path: s3://prod-tpgt-knowledge-lake-sandpit-v1/zihan-research/dialogue-state-tracking/PPTOD-AL/Select-Turn/PPTOD-base-LeastConfidence/mwz20/checkpoints


In [3]:
# specify the path to store the model outputs in S3
output_path = f's3://{bucket}/{prefix}/{base_job_name}/model'
output_path

's3://prod-tpgt-knowledge-lake-sandpit-v1/zihan-research/dialogue-state-tracking/PPTOD-AL/Select-Turn/PPTOD-base-LeastConfidence/mwz20/model'

In [4]:
# specify the path to store the source code in S3
code_location = f's3://{bucket}/{prefix}/{base_job_name}/source_code'
code_location

's3://prod-tpgt-knowledge-lake-sandpit-v1/zihan-research/dialogue-state-tracking/PPTOD-AL/Select-Turn/PPTOD-base-LeastConfidence/mwz20/source_code'

In [5]:
# tensorboard_s3_path = f's3://{bucket}/{prefix}/{base_job_name}/tensorboard'
# tensorboard_s3_path

In [6]:
# train_data_path = f's3://{bucket}/{prefix}/Data/MultiWOZ/dst/train_v2.0.json'
# train_data_path

In [7]:
# validation_data_path = f's3://{bucket}/{prefix}/Data/MultiWOZ/dst/dev_v2.0.json'
# validation_data_path

In [8]:
# test_data_path = f's3://{bucket}/{prefix}/Data/MultiWOZ/dst/test_v2.0.json'
# test_data_path

In [9]:
# import os
# import subprocess

# instance_type = 'local'

# if subprocess.call('nvidia-smi') == 0:
#     ## Set type to GPU if one is present
#     instance_type = 'local_gpu'
    
# print("Instance type = " + instance_type)

### Upload the data
We use the ```sagemaker.Session.upload_data``` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [10]:
# sagemaker_session.upload_data(path='data/multiwoz/data/multi-woz-fine-processed', bucket=bucket, key_prefix=f'{prefix}/Data/MWZ20')

## Create a training job using the sagemaker.PyTorch estimator

The `PyTorch` class allows us to run our training function on SageMaker. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. For local training with GPU, we could set this to "local_gpu".  In this case, `instance_type` was set above based on your whether you're running a GPU instance.

After we've constructed our `PyTorch` object, we fit it using the data we uploaded to S3. Even though we're in local mode, using S3 as our data source makes sense because it maintains consistency with how SageMaker's distributed, managed training ingests data.


In [11]:
# inputs = {
#     'train': train_data_path,
#     'validation': validation_data_path,
#     'test': test_data_path
# }

# copy all content in the 'Data' folder from S3 to local
inputs = f's3://{bucket}/{prefix}/Data/MWZ20'
inputs

's3://prod-tpgt-knowledge-lake-sandpit-v1/zihan-research/dialogue-state-tracking/PPTOD-AL/Data/MWZ20'

In [12]:
# hyperparameters = {
#     'config': './configs/KAGE_GPT2_SparseSupervision.jsonnet',
#     'mode': 'train',
#     'experiment_name': 'KAGE-DS-L4P4K2-LastTurn-RandomTurn',
#     'num_layer': 4,
#     'num_head': 4,
#     'num_hop': 2,
#     'graph_mode': 'part',
# #     'only_last_turn': True,
# #     'dummy_dataloader': True
# }

In [13]:
# !df -h

In [14]:
# ecr_image = '763104351884.dkr.ecr.ap-southeast-2.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu110-ubuntu16.04'

In [15]:
# from sagemaker.pytorch import PyTorch
# from sagemaker.debugger import TensorBoardOutputConfig

# # hyperparameters = {'epochs': 2}

# tensorboard_output_config = TensorBoardOutputConfig(
#     s3_output_path=tensorboard_s3_path,
#     container_local_output_path='/opt/ml/output/tensorboard'
# )

# cifar10_estimator = PyTorch(entry_point='main.py',
#                             source_dir='Src',
#                             output_path=output_path,
#                             code_location=code_location,
#                             role=role,
#                             image_uri=ecr_image,
# #                             framework_version='1.6',
# #                             py_version='py3',
# #                             volume_size=256,
#                             hyperparameters=hyperparameters,
#                             instance_count=1,
#                             instance_type=instance_type,
#                             base_job_name=base_job_name,
#                             tensorboard_output_config=tensorboard_output_config
#                            )

# cifar10_estimator.fit(inputs)

## Run a baseline training job on SageMaker

Now we run training jobs on SageMaker, starting with our baseline training job.

Once again, we create a PyTorch estimator, with a couple key modfications from last time:

* `instance_type`: the instance type for training. We set this to `ml.p3.2xlarge` because we are training on SageMaker now. For a list of available instance types, see [the AWS documentation](https://aws.amazon.com/sagemaker/pricing/instance-types).
* `metric_definitions`: the metrics (defined above) that we want sent to CloudWatch.

In [16]:
use_spot_instances = True
max_run = 120 * 60 * 60 # max run time is 120hours
# max_run = 240 * 60 * 60 # max run time is 120hours
max_wait = 90000 if use_spot_instances else None # max waiting 20mins

In [17]:
ecr_image = '763104351884.dkr.ecr.ap-southeast-2.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu110-ubuntu16.04'
# ecr_image = '763104351884.dkr.ecr.ap-southeast-2.amazonaws.com/pytorch-training:1.7.1-gpu-py36-cu110-ubuntu18.04'

In [18]:
from sagemaker.pytorch import PyTorch
from sagemaker.debugger import TensorBoardOutputConfig


# tensorboard_output_config = TensorBoardOutputConfig(
#     s3_output_path=tensorboard_s3_path,
#     container_local_output_path='/opt/ml/output/tensorboard'
# )

random_seed = random.randint(0, 999)
print(random_seed)

hyperparameters = {
    'model_name': 't5-base',
    'gradient_accumulation_steps': 4,
    'number_of_gpu': 1,
    'batch_size_per_gpu': 2,
    'train_data_ratio': 1.0,
    'random_seed': random_seed,
    
#     'acquisition': 'random',
    'acquisition': 'least_confidence',
#     'acquisition': 'max_entropy',
    
    'num_dialogue_per_round': 100,
    'pre_selected_turn_path': './S123_selected_turn_id.csv',
    
    'patience': 3,
    'epoch_num': 50,
    
    'only_last_turn': 'False',
    'only_random_turn': 'False',
    'data_path_prefix': './mwz_data/mwz20/multi-woz-fine-processed/', # MWZ2.0
    'pretrained_path': './pretrained_pptod/base',
    'ckpt_save_path': f'/opt/ml/model',
    
    
#     'config': './configs/KAGE_GPT2_SparseSupervision.jsonnet',
#     'mode': 'train',
#     'selected_turn_path': './selected_turns/selected_turn_id.csv', # csv file path
    
    # only for continue training
#     'continue_select_turn': 1,
#     'selected_turn_path': './selected_turns', # folder contains selected_turn_id.csv, model_best.pth.tar, tokenizer/
    
#     'config': './configs/KAGE_GPT2_FullTraining.jsonnet',
#     'mode': 'select_turn',
#     'budget': 0,
#     'budget_random_turn': 0, # use 0 as the last turn to train
#     'budget_epoch': 5,
#     'round_epoch': 150, # for each round, train 80 epochs and apply early stop
    
#     'acquisition': 'random',
# #     'acquisition': 'last_turn',
# #     'acquisition': 'least_confidence',
# #     'acquisition': 'max_entropy',
    
#     ###### Remeber to go to main.py to change "--only_last_turn" to False !!!
    
#     'experiment_name': base_job_name,
#     'num_layer': 4,
#     'num_head': 4,
#     'num_hop': 2,
#     'graph_mode': 'part',
#     'random_seed': random_seed
}

environment = {
    'ENV': 'sagemaker_training_job'
}

# metric_definitions = [
#     {
#         'Name': 'epoch',
#         'Regex': 'epoch (.*?) - '
#     },
#     {
#         'Name': 'train_loss',
#         'Regex': 'train loss (.*?)='
#     },
#     {
#         'Name': 'valid_joint_acc',
#         'Regex': 'joint_acc: (.*?) '
#     },
#     {
#         'Name': 'valid_slot_acc',
#         'Regex': 'slot_acc: (.*?) '
#     },
#     {
#         'Name': 'test_joint_acc',
#         'Regex': 'test joint_acc: (.*?) '
#     }
# ]

estimator = PyTorch(
#     entry_point='learn.py',
    
    entry_point='active_learn.py',
    
    source_dir='DST',
    output_path=output_path,
    code_location=code_location,
    role=role,
    image_uri=ecr_image,
#                             framework_version='1.6',
#                             py_version='py3',
    hyperparameters=hyperparameters,
    instance_count=1,
    instance_type='ml.g4dn.xlarge', 
#                             instance_type='ml.p3.2xlarge', # V100
#                             instance_type='ml.p3.2xlarge',

#                             base_job_name=f'Debug-train500-S{random_seed}',
#                             base_job_name=f'Rand-B24D2000Bud0-S{random_seed}',
    
#     base_job_name=f'PPTOD-MWZ20-ME-D100-S{random_seed}',
#     base_job_name=f'PPTOD-MWZ20-Rand-D100-S{random_seed}',
    base_job_name=f'PPTOD-MWZ20-LC-D100-S{random_seed}',
    
#     base_job_name=f'PPTOD-MWZ20-Rand-B24D2000Bud0-S{random_seed}',
#     base_job_name=f'PPTOD-MWZ20-LC-B24D2000Bud0-S{random_seed}',
#     base_job_name=f'PPTOD-MWZ20-ME-B24D2000Bud0-S{random_seed}',
    
#                             base_job_name=f'Train-PPTOD-base-Rand-S{random_seed}',

#                             base_job_name=f'Train-KAGE-LT-MWZ21-S{random_seed}',
    # base_job_name=f'Train-B16D2000Bud0-S{random_seed}',


    # spot training settings: Remember also to set use_spot_instances=True in main.py
#                             checkpoint_s3_uri=checkpoint_s3_path,
    debugger_hook_config=False,
#                             use_spot_instances=use_spot_instances,
    max_run=max_run, # if not set, default is 86400=24h
#                             max_wait=max_wait,
#                             volume_size=225,
    volume_size=125,
    # tensorboard setting
#                             tensorboard_output_config=tensorboard_output_config,
#                             metric_definitions=metric_definitions,
    environment=environment
   )

# estimator.fit(inputs)
estimator.fit()

43
2022-06-19 01:22:56 Starting - Starting the training job...
2022-06-19 01:23:20 Starting - Preparing the instances for trainingProfilerReport-1655601776: InProgress
......
2022-06-19 01:24:21 Downloading - Downloading input data...
2022-06-19 01:24:41 Training - Downloading the training image..................
2022-06-19 01:27:42 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-06-19 01:27:44,952 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-06-19 01:27:44,974 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-06-19 01:27:44,983 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-06-19 01:29:24,823 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python -m pip install -r requiremen

KeyboardInterrupt: 

In [ ]:
from sagemaker.analytics import TrainingJobAnalytics

analysis = TrainingJobAnalytics(training_job_name=estimator._current_job_name)
df = analysis.dataframe()
df

In [ ]:
trained_model_s3_path = cifar10_estimator.model_data
trained_model_s3_path

In [ ]:
output_s3_path = trained_model_s3_path[trained_model_s3_path.find(base_job_name):trained_model_s3_path.rfind('/')+1] + 'output.tar.gz'
output_s3_path

In [ ]:
f'{prefix}/{output_s3_path}'

In [ ]:
# download the output.tar.gz
sagemaker_session.download_data(path='checkpoints', bucket=bucket, key_prefix=f'{prefix}/{output_s3_path}')

In [ ]:
# list the content of the tar file
!tar -ztvf checkpoints/output.tar.gz

In [ ]:
%%time
# Extract the files from output.tar.gz to checkpoints directory
!tar -zxvf checkpoints/output.tar.gz -C checkpoints

In [ ]:
tensorboard_s3_output_path = cifar10_estimator.latest_job_tensorboard_artifacts_path()
tensorboard_s3_output_path

In [ ]:
!pip uninstall tensorboard -y

In [ ]:
# version 2.4.1 works
!pip install tensorboard==2.3.0

In [ ]:
!AWS_REGION=ap-southeast-2 tensorboard --logdir $tensorboard_s3_output_path

# Managed Spot Training with a PyTorch Estimator

For Managed Spot Training using a PyTorch Estimator we need to configure two things:
1. Enable the `train_use_spot_instances` constructor arg - a simple self-explanatory boolean.
2. Set the `train_max_wait` constructor arg - this is an int arg representing the amount of time you are willing to wait for Spot infrastructure to become available. Some instance types are harder to get at Spot prices and you may have to wait longer. You are not charged for time spent waiting for Spot infrastructure to become available, you're only charged for actual compute time spent once Spot instances have been successfully procured.

Normally, a third requirement would also be necessary here - modifying your code to ensure a regular checkpointing cadence - however, PyTorch Estimators already do this, so no changes are necessary here. Checkpointing is highly recommended for Manage Spot Training jobs due to the fact that Spot instances can be interrupted with short notice and using checkpoints to resume from the last interruption ensures you don't lose any progress made before the interruption.

Feel free to toggle the `use_spot_instances` variable to see the effect of running the same job using regular (a.k.a. "On Demand") infrastructure.

Note that `max_wait` can be set if and only if `use_spot_instances` is enabled and **must** be greater than or equal to `max_run`.

In [ ]:
use_spot_instances = True
max_run=600
max_wait = 1200 if use_spot_instances else None

## Simulating Spot interruption after 5 epochs

Our training job should run on 10 epochs.

However, we will simulate a situation that after 5 epochs a spot interruption occurred.

The goal is that the checkpointing data will be copied to S3, so when there is a spot capacity available again, the training job can resume from the 6th epoch.

Note the `checkpoint_s3_uri` variable which stores the S3 URI in which to persist checkpoints that the algorithm persists (if any) during training.

The `debugger_hook_config` parameter must be set to `False` to enable checkpoints to be copied to S3 successfully.

In [ ]:
hyperparameters = {'epochs': 5}


spot_estimator = PyTorch(entry_point='source_dir/cifar10.py',
                            role=role,
                            framework_version='1.7.1',
                            py_version='py3',
                            instance_count=1,
                            instance_type='ml.p3.2xlarge',
                            base_job_name='cifar10-pytorch-spot-1',
                            hyperparameters=hyperparameters,
                            checkpoint_s3_uri=checkpoint_s3_path,
                            debugger_hook_config=False,
                            use_spot_instances=use_spot_instances,
                            max_run=max_run,
                            max_wait=max_wait)

spot_estimator.fit(inputs)

### Savings
Towards the end of the job you should see two lines of output printed:

- `Training seconds: X` : This is the actual compute-time your training job spent
- `Billable seconds: Y` : This is the time you will be billed for after Spot discounting is applied.

If you enabled the `use_spot_instances` var then you should see a notable difference between `X` and `Y` signifying the cost savings you will get for having chosen Managed Spot Training. This should be reflected in an additional line:
- `Managed Spot Training savings: (1-Y/X)*100 %`

### View the job training Checkpoint configuration
We can now view the Checkpoint configuration from the training job directly in the SageMaker console.

Log into the [SageMaker console](https://console.aws.amazon.com/sagemaker/home), choose the latest training job, and scroll down to the Checkpoint configuration section. 

Choose the S3 output path link and you'll be directed to the S3 bucket were checkpointing data is saved.

You can see there is one file there:

```python
checkpoint.pth
```

This is the checkpoint file that contains the epoch, model state dict, optimizer state dict, and loss.

### Continue training after Spot capacity is resumed

Now we simulate a situation where Spot capacity is resumed.

We will start a training job again, this time with 10 epochs.

What we expect is that the tarining job will start from the 6th epoch.

This is done when training job starts. It checks the checkpoint s3 location for checkpoints data. If there are, they are copied to `/opt/ml/checkpoints` on the training conatiner.

In the code you can see the function to load the checkpoints data:

```python
def _load_checkpoint(model, optimizer, args):
    print("--------------------------------------------")
    print("Checkpoint file found!")
    print("Loading Checkpoint From: {}".format(args.checkpoint_path + '/checkpoint.pth'))
    checkpoint = torch.load(args.checkpoint_path + '/checkpoint.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch_number = checkpoint['epoch']
    loss = checkpoint['loss']
    print("Checkpoint File Loaded - epoch_number: {} - loss: {}".format(epoch_number, loss))
    print('Resuming training from epoch: {}'.format(epoch_number+1))
    print("--------------------------------------------")
    return model, optimizer, epoch_number
```


In [ ]:
hyperparameters = {'epochs': 10}


spot_estimator = PyTorch(entry_point='source_dir/cifar10.py',
                            role=role,
                            framework_version='1.7.1',
                            py_version='py3',
                            instance_count=1,
                            instance_type='ml.p3.2xlarge',
                            base_job_name='cifar10-pytorch-spot-2',
                            hyperparameters=hyperparameters,
                            checkpoint_s3_uri=checkpoint_s3_path,
                            debugger_hook_config=False,
                            use_spot_instances=use_spot_instances,
                            max_run=max_run,
                            max_wait=max_wait)

spot_estimator.fit(inputs)

### Analyze training job logs

Analyzing the training job logs, we can see that now, the training job starts from the 6th epoch.

We can see the output of `_load_checkpoint` function:

```
--------------------------------------------
Checkpoint file found!
Loading Checkpoint From: /opt/ml/checkpoints/checkpoint.pth
Checkpoint File Loaded - epoch_number: 5 - loss: 0.8455273509025574
Resuming training from epoch: 6
--------------------------------------------
```

### View the job training Checkpoint configuration after job completed 10 epochs

We can now view the Checkpoint configuration from the training job directly in the SageMaker console.  

Log into the [SageMaker console](https://console.aws.amazon.com/sagemaker/home), choose the latest training job, and scroll down to the Checkpoint configuration section. 

Choose the S3 output path link and you'll be directed to the S3 bucket were checkpointing data is saved.

You can see there is still that one file there:

```python
checkpoint.pth
```

You'll be able to see that the date of the checkpoint file was updated to the time of the 2nd Spot training job.

# Deploy the trained model to prepare for predictions

The deploy() method creates an endpoint which serves prediction requests in real-time.

In [ ]:
from sagemaker.pytorch import PyTorchModel

predictor = spot_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

# Invoking the endpoint

In [ ]:
# get some test images
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%4s' % classes[labels[j]] for j in range(4)))

outputs = predictor.predict(images.numpy())

_, predicted = torch.max(torch.from_numpy(np.array(outputs)), 1)

print('Predicted: ', ' '.join('%4s' % classes[predicted[j]]
                              for j in range(4)))

# Clean-up

To avoid incurring extra charges to your AWS account, let's delete the endpoint we created:

In [ ]:
predictor.delete_endpoint()